In [1]:
import sys
import numpy as np

from sklearn.metrics import mean_squared_error

sys.path.append('..')
from higgs_inference.various.utils import format_number

In [2]:
result_dir = '../results/'

# TablePrinter class

In [8]:
class TablePrinter:
    
    def __init__(self, metric_fns=[], header=None, precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []
        self.block_brackets = []

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = ''
        if header is not None:
            self.table += self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, (line, brackets) in enumerate(zip(self.block_entries, self.block_brackets)):
            
            # Skip entirely empty lines
            if not np.any(np.isfinite(line[2:])):
                continue
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    if brackets[j+2]:
                        text += '(' + format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j])) + ')'
                    else:
                        text += format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
        self.block_brackets = []
    
    
    def new_block(self):
        self.finalise_block()
        self.table += self.indent + self.midrule + self.end_line
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        brackets = [False, False]
        
        # Metrics
        for fn in self.metric_fns:
            bracket = False
            try:
                value = fn(filename, folder)
            except (IOError, ValueError):
                #print('File', filename, 'in folder', folder, 'not found')
                value = np.nan
                
            if isinstance(value, (list, tuple)):
                value, bracket = value
                
            line.append(value)
            brackets.append(bracket)

        self.block_entries.append(line)
        self.block_brackets.append(brackets)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [9]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def median_error_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.sqrt(np.median(errors))

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    try:
        return np.sqrt(mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def median_error_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.median(errors)

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    try:
        return np.sqrt(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return np.sqrt(mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True
    
def median_error_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    errors = np.abs(expected_log_r_truth - expected_log_r_estimated)
    return np.median(errors)

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    try:
        return np.sqrt(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return np.sqrt(mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True
    
def median_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    errors = np.abs(expected_log_r_truth - expected_log_r_estimated)
    return np.median(errors)

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return np.sqrt(mean_squared_error(t_truth, t_estimated))
    
def median_error_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return np.sqrt(mean_squared_error(t_truth, t_estimated))
    
def median_error_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def var_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    return np.sqrt(np.var(expected_log_r_truth - expected_log_r_estimated))

def var_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return np.sqrt(np.var(expected_log_r_truth - expected_log_r_estimated))

# Main tables

In [10]:
labels = [r'AFC (density est.\ in $\boldx$)',
          'carl (PbP, raw)',
          'carl (PbP, cal.)',
          'carl (param., raw)',
          'carl (param., cal.)',
          'carl + score (param., raw)',
          'carl + score (param., cal.)',
          'SM score regression',
          'Ratio regression (PbP, raw)',
          'Ratio regression (param., raw)',
          r'Ratio regr.\ + score (param., raw)']

folders = (['afc']
          + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['score_regression']
          + ['point_by_point'] + ['parameterized'] * 2)

filenames = ['afc',
             'carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'combined',
             'combined_calibrated',
             'scoreregression',
             'regression',
             'regression',
             'combinedregression']


def show_main_table(algorithm_begin=0, algorithm_end=None):
    
    table = TablePrinter([mse_log_r_nottrained, median_error_log_r_nottrained,
                          mse_expected_log_r, median_error_expected_log_r],
                          precisions=[2,2,1,1])

    for i, (label, filename, folder) in enumerate(
        zip(labels[algorithm_begin:algorithm_end], filenames[algorithm_begin:algorithm_end], folders[algorithm_begin:algorithm_end])):

        if i > 0:
            table.new_block()

        if folder == 'point_by_point':
            #table.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
            table.add(label, 'PbP (2)', filename, 'point_by_point')
            #table.add('', 'PbP (3)', filename + '_deep', 'point_by_point')

        elif folder == 'afc':
            table.add(label, '2d, $\epsilon = 0.55$ (2)', filename + '_2d', 'afc')
            table.add(label, '2d, $\epsilon = 0.2$ (2)', filename + '_2d_epsilon_0.20', 'afc')

            table.add(label, '5d, $\epsilon = 0.2$ (2)', filename + '_5d', 'afc')
            table.add(label, '5d, $\epsilon = 0.1$ (2)', filename + '_5d_epsilon_0.10', 'afc')

        elif folder == 'score_regression':
            table.add(label, r'Fixed density est. on $t$  (2)', filename + '_score', 'score_regression')
            table.add('', r'Dyn. dens. est. on $t$  (2)', filename + '_rotatedscore', 'score_regression')
            table.add('', r'Dens. est. on $t \cdot \theta$  (2)', filename + '_scoretheta', 'score_regression')

        else:
            table.add(label, 'Baseline (2)', filename)
            table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
            
            table.add('', 'Aware, baseline (2)', filename + '_aware')
            table.add('', 'Aware, baseline, SM dev. (2)', filename + '_aware_factorsm')
            table.add('', 'Aware, baseline, small LR (2)', filename + '_aware_slowlearning')
            table.add('', 'Aware, baseline, small LR, SM dev. (2)', filename + '_aware_factorsm_slowlearning')
            table.add('', 'Aware, basis (2)', filename + '_aware_basis')

    print(table.print())

In [11]:
show_main_table(0,5)

   AFC (density est.\ in $\boldx$) & 2d, $\epsilon = 0.55$ (2) & \emph{0.46} & \emph{0.28} & \emph{5.2} & \emph{2.5}\\
   AFC (density est.\ in $\boldx$) & 5d, $\epsilon = 0.2$ (2) & 0.57 & 0.33 & 5.7 & 3.8\\
   \midrule
   carl (PbP, raw) & PbP (2) & \emph{0.38} & \emph{0.50} & \emph{17.7} & \emph{17.2}\\
   \midrule
   carl (PbP, cal.) & PbP (2) & \emph{0.27} & \emph{0.24} & \emph{1.4} & \emph{0.5}\\
   \midrule
   carl (param., raw) & Baseline (2) & \emph{0.35} & 0.41 & \emph{3.7} & \emph{3.2}\\
    & Random $\boldtheta$ (2) & 0.36 & \emph{0.36} & 8.3 & 7.7\\
    & Aware, baseline (2) &  &  & (9.7) & \\
   \midrule
   carl (param., cal.) & Baseline (2) & \emph{0.32} & \emph{0.24} & 2.6 & 1.5\\
    & Random $\boldtheta$ (2) & 0.35 & 0.27 & \emph{1.6} & \emph{0.7}\\



/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/ipykernel/__main__.py:12: RuntimeWarning: divide by zero encountered in log


In [12]:
show_main_table(5,8)

   carl + score (param., raw) & Baseline (2) & \emph{0.20} & \emph{0.28} & 23.9 & 23.8\\
    & Random $\boldtheta$ (2) & 0.53 & 0.52 & 22.1 & 20.7\\
    & Aware, baseline (2) &  &  & (\emph{8.3}) & \emph{6.3}\\
   \midrule
   carl + score (param., cal.) & Baseline (2) & \emph{0.18} & \emph{0.21} & \emph{1.9} & 1.7\\
    & Random $\boldtheta$ (2) & 0.38 & 0.24 & 3.9 & \emph{1.2}\\
   \midrule
   SM score regression & Fixed density est. on $t$  (2) & 0.19 & 0.19 & \emph{1.7} & \emph{1.2}\\
    & Dyn. dens. est. on $t$  (2) & 0.19 & 0.19 & 1.8 & 1.2\\
    & Dens. est. on $t \cdot \theta$  (2) & \emph{0.19} & \emph{0.16} & 3.1 & 2.0\\



In [13]:
show_main_table(8,None)

   Ratio regression (PbP, raw) & PbP (2) & \emph{0.09} & \emph{0.17} & \emph{3.3} & \emph{2.0}\\
   \midrule
   Ratio regression (param., raw) & Baseline (2) & 0.15 & 0.16 & 3.8 & 1.0\\
    & Random $\boldtheta$ (2) & \emph{0.13} & \emph{0.16} & \emph{0.7} & \emph{0.4}\\
    & Aware, baseline (2) & 0.68 & 0.22 & 5.7 & 3.3\\
    & Aware, basis (2) & 8.33 & 3.07 & 271.0 & 112.9\\
   \midrule
   Ratio regr.\ + score (param., raw) & Baseline (2) & \emph{0.13} & \emph{0.16} & 1.5 & 1.0\\
    & Random $\boldtheta$ (2) & 0.21 & 0.17 & \emph{1.1} & \emph{0.7}\\
    & Aware, baseline (2) &  &  & 14.0 & 5.7\\



# Score accuracy table

In [14]:
labels = ['carl',
          #'Score',
          'carl + score',
          'Ratio regression',
          'Ratio regr.\ + score']

filenames = ['carl',
             #'score',
             'combined',
             'regression',
             'combinedregression']

table4 = TablePrinter([mse_score_nottrained, median_error_score_nottrained],
                      precisions=[1,2])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table4.new_block()
        
    #table4.add(label, 'Baseline (1)', filename + '_shallow')
    table4.add(label, 'Baseline (2)', filename)
    #table4.add('', 'Baseline (3)', filename + '_deep')
    
    #table4.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
    table4.add('', r'Random $\boldtheta$ (2)', filename + '_random')
    #table4.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

    #table4.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
    table4.add('', 'Aware, baseline (2)', filename + '_aware')
    #table4.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

    #table4.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
    table4.add('', 'Aware, basis (2)', filename + '_aware_basis')
    #table4.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table4.print())

   carl & Baseline (2) & 27.6 & \emph{0.25}\\
    & Random $\boldtheta$ (2) & \emph{27.5} & 0.31\\
   \midrule
   carl + score & Baseline (2) & \emph{27.5} & \emph{0.06}\\
    & Random $\boldtheta$ (2) & 27.6 & 0.10\\
   \midrule
   Ratio regression & Baseline (2) & 27.5 & \emph{0.09}\\
    & Random $\boldtheta$ (2) & \emph{27.5} & 0.09\\
    & Aware, baseline (2) & 29.3 & 0.14\\
    & Aware, basis (2) & 32.5 & 0.22\\
   \midrule
   Ratio regr.\ + score & Baseline (2) & \emph{27.5} & 0.07\\
    & Random $\boldtheta$ (2) & 27.5 & \emph{0.06}\\

